In [ ]:
# imports

In [ ]:
# data set kiezen en inlezen
# representatie van de data zodat die makkelijk terug te halen

In [ ]:
# query preprocessing

In [ ]:
# retrieval model with ranking function to compare all docs and query

In [ ]:
# display results, sowieso, nog interface hierop knallen aan het einde lijkt me?